> Keys will be provided during the workshop

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

cohere_api_key = os.getenv("COHERE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
weaviate_cloud_url = os.getenv("WIKI_DEMO_URL")
weaviate_cloud_key = os.getenv("WIKI_DEMO_KEY_READ")

In [ ]:
import weaviate, os

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_cloud_url,
    auth_credentials=weaviate.auth.AuthApiKey(weaviate_cloud_key),

    headers={
        "X-OpenAI-Api-Key": openai_api_key,
        "X-Cohere-Api-Key": cohere_api_key,
    }
)

client.is_ready()

In [ ]:
wiki = client.collections.get("Wikipedia")

# TODO: write near text query to search for "How do planes fly"
# TODO: return properties "text" and "title"
response = wiki.query.

for item in response.objects:
    print(item.properties)

In [ ]:
wiki = client.collections.get("Wikipedia")

prompt="Explain what this is about? {text}"

# TODO: implement near text on generate
# TODO: provide the prompt as single_prompt
response = wiki.generate.

print("=== Source ===")
for item in response.objects:
    print(item.properties)
    print(item.generated)

In [ ]:
wiki = client.collections.get("Wikipedia")

task = "Explain, how do planes fly? Please only use the provided content."

# TODO: implement near text on generate
# TODO: provide the task as grouped_task
response = wiki.generate.

print("=== Generated Response ===")
print(response.generated)

print("=== Source ===")
for item in response.objects:
    print(item.properties)

In [ ]:
wiki = client.collections.get("Wikipedia")



response = wiki.generate.near_text(
    query="How do planes fly",
    auto_limit=1,
    grouped_task="Explain, how do planes fly? Please only use the provided content.",
    # TODO: add grouped properties to only provide "text" and "title"
)

print("=== Generated Response ===")
print(response.generated)

print("=== Source ===")
for item in response.objects:
    print(item.properties)

In [ ]:
# TODO: You can call both single_prompt and grouped_task together
# TODO: Feel free to figure it out how